In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import transforms
import torch.optim as optim

torch.set_grad_enabled(True)
train_set=torchvision.datasets.FashionMNIST(root='/kaggle/working/data',train=True,download=True,transform=transforms.Compose([transforms.ToTensor()]))

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.net=nn.Sequential(
            
        nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5),
        nn.BatchNorm2d(num_features=6),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),
            
        nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5),
        nn.BatchNorm2d(num_features=12),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),
            
        nn.Flatten(),
        nn.Linear(in_features=12*4*4,out_features=120),
        nn.ReLU(inplace=True),
        nn.Linear(in_features=120,out_features=60),
        nn.ReLU(inplace=True),
        nn.Linear(in_features=60,out_features=10),
        
        nn.Softmax(dim=1)
        )
        
    def forward(self,t):
        for layer in self.net:
            t=layer(t)
        return t


In [ ]:
model=Network()
EPOCHS=1
BATCH_SIZE=128
LEARNING_RATE=0.001
train_loader=torch.utils.data.DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True,pin_memory=True)
optimizer=optim.Adam(model.parameters(),lr=LEARNING_RATE)

In [ ]:
for epoch in range(EPOCHS):
    total_loss=0.0
    for batch in train_loader:
        X,y=batch
        y_test=model(X)
        loss=F.cross_entropy(y_test,y)
        optimizer.zero_grad()
        total_loss+=loss.item()
        loss.backward()
        optimizer.step()
    print("Epoch : {0} ---------- {1}".format(epoch,total_loss*BATCH_SIZE/len(train_set)))

In [ ]:
with torch.no_grad():
    total_correct=0
    all_preds=torch.tensor([])
    loader=torch.utils.data.DataLoader(train_set,batch_size=1000)
    for batch in loader:
            X,y=batch
            y_test=model(X)
            total_correct+=y_test.argmax(dim=1).eq(y).sum().item()
            all_preds=torch.cat((all_preds,y_test),dim=0)
    print(total_correct,total_correct/60000)
    all_preds=all_preds.argmax(dim=1).cpu()

In [ ]:
from sklearn.metrics import confusion_matrix as cm
con_m=cm(train_set.targets,all_preds)
plot_confusion_matrix(con_m,set(train_set.classes))

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
